In [2]:
#! /usr/bin/env python3.6
import requests
import datetime
from bs4 import BeautifulSoup
import time
import pandas as pd
import re
import math
import pickle

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import numpy as np
from IPython.display import clear_output
from selenium.webdriver.common.by import By

import sys
sys.path.append("../")
import src.soporteScrapping as scr
import warnings
warnings.filterwarnings('ignore')


new

In [3]:
country = 'Sicily'
rooms = 1
adults = 2
checkin = '2023-09-01'
checkout = '2023-09-08'

In [4]:
list_url = []
list_dates = []
for i in range(1,30):
    if (len(str(i)) == 1) & (len(str(i+1)) == 1):
        day_in = '0' + str(i)
        day_out = '0' + str(i+1)
    elif (len(str(i)) == 1) & (len(str(i+1)) == 2):
        day_in = '0' + str(i)
        day_out = str(i+1)
    else:
        day_in = str(i)
        day_out = str(i+1)
    url_original_test = scr.url(rooms, country, f'2023-09-{day_in}', f'2023-09-{day_out}', adults)
    list_dates.append(f'2023-09-{day_in}')
    list_url.append(url_original_test)
list_url[0]

'https://www.booking.com/searchresults.en-gb.html?ss=Sicily%2C+Italy&ssne=Sicily%2C+Italy&ssne_untouched=Sicily%2C+Italy&efdco=1&label=gen173nr-1DCAEoggJCAlhYSDNYBGiTAYgBAZgBLsIBCndpbmRvd3MgMTDIAQzYAQPoAQGSAgF5qAID&sid=716ea5d78c4043fd78b7a1410d639e3d&aid=304142&lang=en-gb&sb=1&src_elem=sb&src=index&dest_id=909&dest_type=region&ac_position=0&ac_click_type=b&ac_langcode=en&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=36746f79352b0064&ac_meta=GhAzNjc0NmY3OTM1MmIwMDY0IAAoATICZW46BFNpY2lAAEoAUAA%3D&checkin=2023-09-01&checkout=2023-09-02&group_adults=2&no_rooms=1&group_children=0&sb_travel_purpose=leisure'

In [15]:
almacen = {
    'url':[],
    'name':[],
    'pagina':[],
    'address':[],
    'currentPrice':[],
    'previousPrice':[],
    'rating':[],
    'reviewNumber':[],
    'chekingDate':[],
    'image':[]
}

hace solo una paorte del bucle

In [16]:
scr.opcionesSelenium()

In [17]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(list_url[0])
driver.maximize_window()
driver.set_window_size(1920, 1080)
driver.implicitly_wait(30)
driver.find_element('css selector', '#onetrust-accept-btn-handler').click()
res = requests.get(list_url[0])
soup_numero = BeautifulSoup(res.content, 'html.parser')
pagina_numero = soup_numero.text
number = math.ceil(int(re.findall(r': (.*) properties', pagina_numero)[0].replace(',', '')) / 25)
try:
    try:
        for i in range(3, 50):
            print('item:', i)
            try:
                try:
                    name = driver.find_element('xpath', '//*[@id="search_results_table"]/div[2]/div/div/div/div[3]/div[{i}]/div[1]/div[2]/div/div[1]/div/div[1]/div/div[1]/div/h3/a/div[1]'.format(i=i)).text
                    previousPrice = driver.find_element('xpath', '//*[@id="search_results_table"]/div[2]/div/div/div/div[3]/div[{i}]/div[1]/div[2]/div/div[3]/div[2]/div/div[1]/span/div/span[1]'.format(i=i)).text
                    currentPrice = driver.find_element('xpath', '//*[@id="search_results_table"]/div[2]/div/div/div/div[3]/div[{i}]/div[1]/div[2]/div/div[3]/div[2]/div/div[1]/span/div/span[2]'.format(i=i)).text
                    rating = driver.find_element('xpath', '//*[@id="search_results_table"]/div[2]/div/div/div/div[3]/div[{i}]/div[1]/div[2]/div/div[1]/div/div[2]/div/div/div/a/span/div/div[1]'.format(i=i)).text
                    reviewNumber = driver.find_element('xpath', '//*[@id="search_results_table"]/div[2]/div/div/div/div[3]/div[{i}]/div[1]/div[2]/div/div[1]/div/div[2]/div/div/div/a/span/div/div[2]/div[2]'.format(i=i)).text
                    image = driver.find_element('xpath', '//*[@id="search_results_table"]/div[2]/div/div/div/div[3]/div[{i}]/div[1]/div[1]/div/a/img'.format(i=i)).get_attribute('src')
                    driver.find_element('xpath', f'//*[@id="search_results_table"]/div[2]/div/div/div/div[3]/div[{i}]/div[1]/div[2]/div/div[1]/div/div[1]/div/div[1]/div/h3/a').click()
                except:
                    name = driver.find_element('xpath', '//*[@id="search_results_table"]/div[2]/div/div/div/div[3]/div[{i}]/div[1]/div[2]/div/div[1]/div/div[1]/div/div[1]/div/h3/a/div[1]'.format(i=i)).text
                    try:
                        previousPrice = driver.find_element('xpath', '//*[@id="search_results_table"]/div[2]/div/div/div/div[3]/div[{i}]/div[1]/div[2]/div/div[3]/div[2]/div/div/span/div/span[1]'.format(i=i)).text
                        currentPrice = driver.find_element('xpath', '//*[@id="search_results_table"]/div[2]/div/div/div/div[3]/div[{i}]/div[1]/div[2]/div/div[3]/div[2]/div/div/span/div/span[2]'.format(i=i)).text
                    except:
                        previousPrice = np.nan
                        currentPrice = driver.find_element('xpath', '//*[@id="search_results_table"]/div[2]/div/div/div/div[3]/div[{i}]/div[1]/div[2]/div/div[2]/div[2]/div/div[1]/span'.format(i=i)).text
                    rating = driver.find_element('xpath', '//*[@id="search_results_table"]/div[2]/div/div/div/div[3]/div[{i}]/div[1]/div[2]/div/div[1]/div/div[2]/div/div/div/a/span/div/div[1]'.format(i=i)).text
                    reviewNumber = driver.find_element('xpath', '//*[@id="search_results_table"]/div[2]/div/div/div/div[3]/div[{i}]/div[1]/div[2]/div/div[1]/div/div[2]/div/div/div/a/span/div/div[2]/div[2]'.format(i=i)).text
                    image = driver.find_element('xpath', '//*[@id="search_results_table"]/div[2]/div/div/div/div[3]/div[{i}]/div[1]/div[1]/div/a/img'.format(i=i)).get_attribute('src')
                    driver.find_element('xpath', f'//*[@id="search_results_table"]/div[2]/div/div/div/div[3]/div[{i}]/div[1]/div[2]/div/div[2]/div[2]/div/div[2]/a').click()
                driver.implicitly_wait(15)
                driver.switch_to.window(driver.window_handles[1])
                url = driver.current_url
                almacen['url'].append(url)
                res = requests.get(url)
                soup = BeautifulSoup(res.content, 'html.parser')
                pagina = soup.text
                almacen['pagina'].append(pagina)
                almacen['address'].append(re.findall('(.*, \d{3,5} .*)', pagina.lower()))
                print('dict:', len(almacen['url']), len(almacen['pagina']))
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
                almacen['name'].append(name)
                almacen['previousPrice'].append(previousPrice)
                almacen['currentPrice'].append(currentPrice)
                almacen['rating'].append(rating)
                almacen['reviewNumber'].append(reviewNumber)
                almacen['image'].append(image)
                print(len(almacen['name']), len(almacen['url']), len(almacen['currentPrice']), len(almacen['previousPrice']))
                clear_output(wait=True)
            except:
                pass
            clear_output
    # driver.find_element('xpath', '//*[@id="search_results_table"]/div[2]/div/div/div/div[4]/div[2]/nav/div/div[3]/button').click()
    # driver.implicitly_wait(2)
    # clear_output(wait=True)
    except:
        pass
except:
    print('me rompí')
    

item: 13


In [9]:
almacen['name'][0]

'Iblaresort Boutique Hotel'

In [10]:
almacen['url'][0]

'https://www.booking.com/hotel/it/iblaresort.en-gb.html?aid=304142&label=gen173nr-1BCAEoggI46AdIM1gEaJMBiAEBmAEJuAEXyAEM2AEB6AEBiAIBqAID2AIB&sid=b2eae561b254ece8e6c54e796529dbf7&all_sr_blocks=42738009_88462426_0_1_0;checkin=2023-09-01;checkout=2023-09-02;dest_id=909;dest_type=region;dist=0;group_adults=2;group_children=0;hapos=1;highlighted_blocks=42738009_88462426_0_1_0;hpos=1;matching_block_id=42738009_88462426_0_1_0;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;sr_pri_blocks=42738009_88462426_0_1_0__12240;srepoch=1675886318;srpvid=b8968c76cfec0361;type=total;ucfs=1&#hotelTmpl'

In [ ]:
print(len(almacen['url']), len(almacen['name']), len(almacen['pagina']), len(almacen['address']), len(almacen['currentPrice']), len(almacen['previousPrice']))

In [ ]:
print(len(list(dict.fromkeys(almacen['name']))))

In [ ]:
re.findall('price(.*)', almacen['pagina'][-2].lower())

In [ ]:
link = 'https://www.booking.com/hotel/it/mer-amp-soleil.en-gb.html?aid=304142&label=gen173nr-1BCAEoggI46AdIM1gEaJMBiAEBmAEJuAEXyAEM2AEB6AEBiAIBqAID2AIB&sid=dc6ef7819845d47e050d4481b6d81df6&all_sr_blocks=197039301_346575045_2_0_0;checkin=2023-09-01;checkout=2023-09-08;dest_id=909;dest_type=region;dist=0;group_adults=2;group_children=0;hapos=24;highlighted_blocks=197039301_346575045_2_0_0;hpos=24;matching_block_id=197039301_346575045_2_0_0;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;sr_pri_blocks=197039301_346575045_2_0_0__79500;srepoch=1675348577;srpvid=e65166afe72c013c;type=total;ucfs=1&#hotelTmpl'


In [ ]:
res = requests.get(link)
soup_numero = BeautifulSoup(res.content, 'html5lib')
#soup_numero.text

hace el bucle completo

In [ ]:
scr.opcionesSelenium()

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url_original)
driver.maximize_window()
driver.set_window_size(1920, 1080)
driver.implicitly_wait(30)
driver.find_element('css selector', '#onetrust-accept-btn-handler').click()
res = requests.get(url_original)
soup_numero = BeautifulSoup(res.content, 'html.parser')
pagina_numero = soup_numero.text
number = math.ceil(int(re.findall(r': (.*) properties', pagina_numero)[0].replace(',', '')) / 25)
try:
    for page in range (1,number):
        try:
            for i in range(2, 50):
                print('item:', i, 'page:', page)
                try:
                    try:
                        name = driver.find_element('xpath', '//*[@id="search_results_table"]/div[2]/div/div/div/div[3]/div[{i}]/div[1]/div[2]/div/div[1]/div/div[1]/div/div[1]/div[1]/h3/a/div[1]'.format(i=i)).text
                        previousPrice = driver.find_element('xpath', '//*[@id="search_results_table"]/div[2]/div/div/div/div[3]/div[{i}]/div[1]/div[2]/div/div[3]/div[2]/div/div[1]/span/div/span[1]'.format(i=i)).text
                        currentPrice = driver.find_element('xpath', '//*[@id="search_results_table"]/div[2]/div/div/div/div[3]/div[{i}]/div[1]/div[2]/div/div[3]/div[2]/div/div[1]/span/div/span[2]'.format(i=i)).text
                        rating = driver.find_element('xpath', '//*[@id="search_results_table"]/div[2]/div/div/div/div[3]/div[{i}]/div[1]/div[2]/div/div[1]/div/div[2]/div/div/a/span/div/div[1]'.format(i=i)).text
                        reviewNumber = driver.find_element('xpath', '//*[@id="search_results_table"]/div[2]/div/div/div/div[3]/div[{i}]/div[1]/div[2]/div/div[1]/div/div[2]/div/div/a/span/div/div[2]/div[2]'.format(i=i)).text
                        image = driver.find_element('xpath', '//*[@id="search_results_table"]/div[2]/div/div/div/div[3]/div[{i}]/div[1]/div[1]/div/a/img'.format(i=i)).get_attribute('src')
                        driver.find_element('xpath', f'//*[@id="search_results_table"]/div[2]/div/div/div/div[3]/div[{i}]/div[1]/div[2]/div/div[3]/div[2]/div/div[2]/a').click()
                    except:
                        name = driver.find_element('xpath', '//*[@id="search_results_table"]/div[2]/div/div/div/div[3]/div[{i}]/div[1]/div[2]/div/div[1]/div/div[1]/div/div[1]/div/h3/a/div[1]'.format(i=i)).text
                        try:
                            previousPrice = driver.find_element('xpath', '//*[@id="search_results_table"]/div[2]/div/div/div/div[3]/div[{i}]/div[1]/div[2]/div/div[3]/div[2]/div/div/span/div/span[1]'.format(i=i)).text
                            currentPrice = driver.find_element('xpath', '//*[@id="search_results_table"]/div[2]/div/div/div/div[3]/div[{i}]/div[1]/div[2]/div/div[3]/div[2]/div/div/span/div/span[2]'.format(i=i)).text
                        except:
                            previousPrice = np.nan
                            currentPrice = driver.find_element('xpath', '//*[@id="search_results_table"]/div[2]/div/div/div/div[3]/div[{i}]/div[1]/div[2]/div/div[2]/div[2]/div/div[1]/span'.format(i=i)).text
                        rating = driver.find_element('xpath', '//*[@id="search_results_table"]/div[2]/div/div/div/div[3]/div[{i}]/div[1]/div[2]/div/div[1]/div/div[2]/div/div/a/span/div/div[1]'.format(i=i)).text
                        reviewNumber = driver.find_element('xpath', '//*[@id="search_results_table"]/div[2]/div/div/div/div[3]/div[{i}]/div[1]/div[2]/div/div[1]/div/div[2]/div/div/a/span/div/div[2]/div[2]'.format(i=i)).text
                        image = driver.find_element('xpath', '//*[@id="search_results_table"]/div[2]/div/div/div/div[3]/div[{i}]/div[1]/div[1]/div/a/img'.format(i=i)).get_attribute('src')
                        driver.find_element('xpath', f'//*[@id="search_results_table"]/div[2]/div/div/div/div[3]/div[{i}]/div[1]/div[2]/div/div[2]/div[2]/div/div[2]/a').click()
                    driver.implicitly_wait(15)
                    driver.switch_to.window(driver.window_handles[1])
                    url = driver.current_url
                    almacen['url'].append(url)
                    res = requests.get(url)
                    soup = BeautifulSoup(res.content, 'html.parser')
                    pagina = soup.text
                    almacen['pagina'].append(pagina)
                    almacen['address'].append(re.findall('(.*, \d{3,5} .*)', pagina.lower()))
                    print('dict:', len(almacen['url']), len(almacen['pagina']))
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])
                    if name in almacen['name']:
                        pass
                    else:
                        almacen['name'].append(name)
                    almacen['previousPrice'].append(previousPrice)
                    almacen['currentPrice'].append(currentPrice)
                    almacen['rating'].append(rating)
                    almacen['reviewNumber'].append(reviewNumber)
                    almacen['image'].append(image)
                    print(len(almacen['name']), len(almacen['url']), len(almacen['currentPrice']), len(almacen['previousPrice']))
                    clear_output(wait=True)
                except:
                    pass
                clear_output
            driver.find_element('xpath', '//*[@id="search_results_table"]/div[2]/div/div/div/div[4]/div[2]/nav/div/div[3]/button').click()
            print(len(almacen['name']), len(almacen['url']), len(almacen['currentPrice']), len(almacen['previousPrice']))
            driver.implicitly_wait(2)
            clear_output(wait=True)
            with open(f'../data/dict_booking.pickle', 'wb') as data_scrapeado:
                pickle.dump(almacen, data_scrapeado)
        except:
            pass
except:
    print('me rompí')
    

In [ ]:
len(almacen['url'])

In [ ]:
almacen['url'][1]

In [ ]:
almacen['url']

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url_original)
driver.maximize_window()
driver.set_window_size(1920, 1080)
driver.implicitly_wait(30)
driver.find_element('css selector', '#onetrust-accept-btn-handler').click()
try:
    #driver.find_elements_by_xpath('')
    driver.find_element('xpath', '//*[@id="search_results_table"]/div[2]/div/div/div/div[3]/div[3]/div[1]/div[2]/div/div[1]/div[1]/div/div[1]/div[1]/h3/a').click().click()
    driver.implicitly_wait(15)
    #almacen['url'].append(driver.getCurrentUrl())
    time.sleep(5)
    for handle in driver.window_handles:
        print('handle')
        driver.switch_to.window(handle)
    #p = driver.window_handles[0]
    #obtain browser tab window
    #c = driver.window_handles[1]
    #switch to tab browser
    #driver.switch_to.window(c)
    try:
        chwd = driver.window_handles
        print(chwd[0])
        for w in chwd:
        #switch focus to child window
            if(w!=p):
                driver.switch_to.window(w)
                page_source = driver.page_source
                time.sleep(1)
            url = driver.current_url
            almacen['url'].append(url)
            res = requests.get(url)
            soup = BeautifulSoup(res.content, 'html.parser')
            pagina = soup.text
            almacen['pagina'].append(pagina)
            #driver.current_url
            #print("The current url is:"+str(driver.current_url))
    except:
        pass
except:
    print('aloha')
    #//*[@id="search_results_table"]/div[2]/div/div/div/div[3]/div[5]/div[1]/div[2]/div/div[1]/div[1]/div/div[1]/div/h3/a
    #//*[@id="search_results_table"]/div[2]/div/div/div/div[3]/div[3]/div[1]/div[2]/div/div[1]/div[1]/div/div[1]/div[1]/h3/a
    #search_results_table > div:nth-child(2) > div > div > div > div.d4924c9e74 > div:nth-child(4) > div.d20f4628d0 > div.b978843432 > div > div.a1b3f50dcd.b2fe1a41c3.a7c67ebfe5.d19ba76520.d14b211b4f > div.b1e6dd8416.aacd9d0b0a > div > div:nth-child(1) > div.dd023375f5 > h3 > a
    #search_results_table > div:nth-child(2) > div > div > div > div.d4924c9e74 > div:nth-child(6) > div.d20f4628d0 > div.b978843432 > div > div.a1b3f50dcd.b2fe1a41c3.a7c67ebfe5.d19ba76520.d14b211b4f > div.b1e6dd8416.aacd9d0b0a > div > div:nth-child(1) > div > h3 > a
#search_results_table > div:nth-chil
# Aur
# d(2) > div > div > div > div.d4924c9e74 > div:nth-child(4) > div.d20f4628d0 > div.b978843432 > div > div.a1b3f50dcd.b2fe1a41c3.a7c67ebfe5.d19ba76520.d14b211b4f > div.b1e6dd8416.aacd9d0b0a > div > div:nth-child(1) > div > h3 > a > div.fcab3ed991.a23c043802
##search_results_table > div:nth-child(2) > div > div > div > div.d4924c9e74 > div:nth-child(6) > div.d20f4628d0 > div.b978843432 > div > div.a1b3f50dcd.b2fe1a41c3.a7c67ebfe5.d19ba76520.d14b211b4f > div.b1e6dd8416.aacd9d0b0a > div > div:nth-child(1) > div > h3 > a > div.fcab3ed991.a23c043802
##search_results_table > div:nth-child(2) > div > div > div > div.d4924c9e74 > div:nth-child(8) > div.d20f4628d0 > div.b978843432 > div > div.a1b3f50dcd.b2fe1a41c3.a7c67ebfe5.d19ba76520.d14b211b4f > div.b1e6dd8416.aacd9d0b0a > div > div:nth-child(1) > div > h3 > a > div.fcab3ed991.a23c043802
#except:
#    driver.find_element('xpath', '//*[@id="__bui-c2718823-1"]').send_keys(city, Keys.TAB)
    

In [ ]:
driver.window_handles[0]

In [ ]:
almacen['url']

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://www.booking.com/')
driver.maximize_window()
driver.set_window_size(1920, 1080)
driver.implicitly_wait(30)
driver.find_element('css selector', '#onetrust-accept-btn-handler').click()
try:
    driver.find_element('xpath', '//*[@id="ss"]').send_keys(city, Keys.TAB)
except:
    driver.find_element('xpath', '//*[@id="__bui-c2718823-1"]').send_keys(city, Keys.TAB)
    

In [ ]:
link = str('https://www.booking.com/searchresults.es.html?'
    'label'
    '=gen173nr-1FCAEoggI46AdIM1gEaEaIAQGYAQq4ARfIAQzYAQHoAQH4AQuIAgGoAgO4AszOv54GwAIB0gIkZjVlYTNmNmQtZGVhOS00MzY5LWFjZDAtZGY4NGVmYmRhNzFm2AIG4AIB&'
    'sid=a4916b536ec3ecd3b62ccc75112c9105&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.es.html%3Flabel%3Dgen173nr-1FCAEoggI46AdIM1gEaEaIAQGYAQq4ARfIAQzYAQHoAQH4AQuIAgGoAgO4AszOv54GwAIB0gIkZjVlYTNmNmQtZGVhOS00MzY5LWFjZDAtZGY4NGVmYmRhNzFm2AIG4AIB%26'
    'sid%3Da4916b536ec3ecd3b62ccc75112c9105%26sb_price'
    '_type%3Dtotal%26%26&ss={country}&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1')

old

In [ ]:
import requests
from scrapy.selector import Selector

# Prepare url
city     = 'Sicily'
main_url = 'https://www.airbnb.com'
city_url = f'{main_url}/s/{city}/homes/'

# Create selector
html = requests.get(city_url).content
sel  = Selector(text=html)

# Get hotels
hotels = sel.css('div._8ssblpx')
print('Number of hotels:', len(hotels))

# Find the next page url
next_page = sel.css('a._za9j7e ::attr(href)').extract_first()
print('Next page:', '\n', next_page)

In [ ]:
def get_booking_page(offset, rooms, country, startdate, enddate):
    """
    Make request to booking page and parse html
    :param offset:
    :return: html page
    """
    checkin: list = str(startdate).split("-")
    checkout: list = str(enddate).split("-")
    url: str = (
        "https://www.booking.com/searchresults.en-gb.html?"
        "aid=304142&label"
        "=gen173nr-1DCAEoggJCAlhYSDNYBGiTAYgBAZgBLsIBCnd"
        "pbmRvd3MgMTDIAQzYAQPoAQGSAgF5qAID&"
        "sid=716ea5d78c4043fd78b7a1410d639e3d&checkin_month="
        "{checkin_m}&checkin_monthday={checkin_d}&checkin_year={checkin_y}&checkout_month={checkout_m}&"
        "checkout_monthday={checkout_d}&checkout_year={checkout_y}"
        "&class_interval=1&dest_id=125&dest_type=country&dtdisc=0&from_sf"
        "=1&genius_rate=1&no_rooms={rooms}&group_adults=2&group_children=0&inac=0&"
        "index_postcard=0&label_click=undef"
        "&no_rooms=1&postcard=0&raw_dest_type=country&room1="
        "A%2CA&sb_price"
        "_type=total&src=searchresults&src_elem=sb&ss={country}&ss_all="
        "0&ssb=empty&sshis=0&ssne={country}"
        "&ssne_untouched={country}&rows=15&offset=".format(
            rooms=rooms,
            country=country.replace(" ", "+"),
            checkin_m=checkin[1],
            checkin_d=checkin[2],
            checkin_y=checkin[0],
            checkout_m=checkout[1],
            checkout_d=checkout[2],
            checkout_y=checkout[0],
        )
        + str(offset)
    )
    r = requests.get(
        url,
        headers={
            "User-Agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0)"
            " Gecko/20100101 Firefox/48.0"
        },
    )
    html = r.content
    parsed_html = BeautifulSoup(html, "lxml")
    return parsed_html